# Self-assembly of functionalized alkanes on a graphite surface

This tutorial guides you how to perform self-assembly simulations on graphite flake using coarse-grained force-field MARTINI [1], similar to work done in publication [2]. Martini force-field was originally developed for lipids [1] and then extended to many other systems including self-assembly on graphite flake [2,3]. For learning purposes, we will limit ourselves to tiny graphite flake with a little number of adsorbent molecules. Such simulations can be done on a personal computer within 2h. The tutorial is prepared for GROMACS version 5 and higher.

This tutorial assumes a basic knowledge of the Linux operating system. It is helpful to have a basic understanding of gromacs molecular dynamics package and Martini Force-Field. You can find tutorials on these topics at http://cgmartini.nl and http://www.bevanlab.biochem.vt.edu/Pages/Personal/justin/gmx-tutorials/lysozyme/index.html. 

In this tutorial, we will perform a simple simulation of self-assembly (that is adsorption and rearrangement) on graphite surface from a random configuration of adsorbent in a solvent. We will study here self-assembly of linear functionalized alkane, AM25, which consist of 6 beads (twp P1 polar and four C1 apolar), which can represent N,N′-decanomethylenebispentamide (C4H9-CO-NH-(CH2)10-NH-CO-C4H9). For more information, look into publication [2]. As a solvent, we use phenyoctane, which in its coarse-grained representation consist of 5 beads (three SC4 beads in a ring and two C1 beads in a tail)


In this tutorial I use python wrapper for gromacs: [GromacsWrapper](https://github.com/Becksteinlab/GromacsWrapper).

In [1]:
import gromacs

##  Setting up the system

Firstly copy sample topology file:

In [2]:
!cp topology/sample_topol.top topol.top

Topology file consist of references to parameters of force-fields and number of molecules in the system. Here is the sample_topol.top file: 

In [3]:
!cat topol.top

#include "FF/martini_v2.15.itp"
#include "FF/martini_v2.0_solvents.itp"
#include "FF/martini_v2.0_graphite.itp"
#include "FF/martini_v2.0_adsorbants.itp"
[ system ]
Adsoption on graphate
[ molecules ]
GRAP 1600
AM25   50
PHEO  300



First four lines describe where parameters of force-field can be found (in subdirectory "FF"), then there is [ system ] with a title of simulation, and then "[ molecule ]", after which there are numbers of molecules used in the simulation. For this tutorial, we use 1600 beads of graphite, 50 molecules of AM25, and 300 molecules of solvent PHEO. The subfolder "topology" also contains topology used in the publication [2].

Copy coordination file of small graphite flake fro "gro" folder (for making of custom graphite flake look at the end of this file).

In [4]:
!cp gro/small_graphite.gro 0_box.gro

And that's how graphite flake 0_box.gro looks like:
![Graphite flake](pictures/0_box.png)
Here, I visualize coordinate file by VMD (you could also use diffrent application, like nglview which can be embedded in jupyter notebook, but coarse-grained simulations require some special treatment)


Insert molecules using "gmx insert-molecules" command: the coordination file for single molecules of adsorbants are in "gro" subdirecotry. Here, we insert 50 molecules: 

In [5]:
gromacs.insert_molecules(f="0_box.gro", ci="gro/AM25.gro", o="0_box_ad.gro", nmol=50)

(0, None, None)

![Graphite with AM25](pictures/0_box_ad.png)

Add solvent molecules using "gmx solvate" command: here we use maxsol command to limit the number of molecules to 300 (however, you don't have to use it, but you have to then make an appropriate change of the number of molecules in topol.top files) 

In [6]:
gromacs.solvate(cp="0_box_ad.gro", cs='gro/phenyloctane.gro', o="0_box_sol.gro", p="topol.top", maxsol=300)

(0, None, None)

![Substrat, adsorbent and solvent](pictures/0_box_sol.png)

## 1. Energy minimalization

After our system is set up, perform energy minimalization, to remove all bad contacts (which could result in the blowup of the system). All parameters files for simulation engine are in "mdp" folder. All *.mdp files are similar to one present on other tutorials of Martini except lines in which we specify that graphite beads are frozen (freezegrps = GRAP;freezedim= Y Y Y) - they do not move during the course of simulation (but they do interact with adsorbent and solvent molecules)  

In [7]:
gromacs.grompp(f="mdp/1_em.mdp", c="0_box_sol.gro", p="topol.top", o="1_em.tpr")
gromacs.mdrun(deffnm ="1_em")

(0, None, None)

Energy minimalization will produce minimized structure in 1_em.gro file, which we use for further simulations.

## 2.Equilibration

We perform the equilibration in two stages: firstly we equilibrate in thermostat (NVT ensemble) and then in barostat (NPT ensemble). Equilibrate system in NVT ensamble:

In [8]:
gromacs.grompp(f="mdp/2_nvt.mdp", c="1_em.gro", p="topol.top", o="2_nvt.tpr", maxwarn=1)
gromacs.mdrun(deffnm ="2_nvt")

(0, None, None)

We use command "maxwarn 1" to ignore one warning:

<i>WARNING 1 [file mdp/2_nvt.mdp]:<br>
  For proper integration of the Berendsen thermostat, tau-t (0.3) should be at least 5 times larger than nsttcouple*dt (0.3)</i>
  
We ignore this warning because we want to have time step as large as possible (to achieve long timescales) and still have control over the temperature. This is not really weak coupled thermostat; however, we make a tradeoff of the accuracy of the simulation for the performance.

Equilibrate system in NPT ensamble:

In [9]:
gromacs.grompp(f="mdp/2_npt.mdp", c="2_nvt.gro", p="topol.top", o="2_npt.tpr", maxwarn=2)
gromacs.mdrun(deffnm ="2_npt")

(0, None, None)

In this case, we use "maxwarn 2" to ignore two  warnings: 

<i>WARNING 1 [file mdp/2_npt.mdp, line 65]:<br>
  All off-diagonal reference pressures are non-zero. Are you sure you want to apply threefold shear stress?</i>

We ignore it because we don't want to stretch simulation box together with graphite flake.

<i>WARNING 2 [file mdp/2_npt.mdp]:<br>
  For proper integration of the Berendsen thermostat, tau-t (0.3) should be at least 5 times larger than nsttcouple*dt (0.3)</i>
  
See above.

## 3. Run simulation

After the temrpature and pressure of the system is set up, it is a time to perform final simulation:

In [11]:
gromacs.grompp(f="mdp/3_run.mdp", c="2_npt.gro", p="topol.top", o="3_run.tpr", maxwarn=2)
gromacs.mdrun(deffnm ="3_run")

(0, None, None)


This command will produce several files, from which final structure is in 3_run.gro and trajectory in 3_run.xtc file. Such a simulation on a PC (CPU Intel(R) Core(TM) i7-5600U CPU @ 2.60GHz) takes about 2h.  You can visualize this trajectory and structure using visualization program such as  VMD (http://www.ks.uiuc.edu/Research/vmd/). 
![Final assembly](pictures/3_run.png)

## 4. Custom size of graphite flake:
You can also make a custom size of the graphite flake from the elementary cell (in gro subfolder file "cell.gro") containing two beads of graphite flake using the command: 

In [ ]:
gromacs.genconf(f="gro/cell.gro", nbox=[200,200,2], o='0_out.gro')


This creates rhombic graphite flake 20x20 in x- and y-direction and four beads in z (the cell contains two beads in the z-direction). Subfolder gro also contains coordination file (graphite_paper.gro) of graphite flake used in the publication [2].

Next, create boudaries simulation box:

In [ ]:
gromacs.editconf(f="0_out.gro", o='0_box.gro', box=[70, 70, 70], angles=[90, 90, 60])

It is essential that the simulation box is larger than the graphite flake (as we can see from the publication, that the infinite graphite flake.

This way you can obtain graphite flake sized as big as you wish, i.g.:
![Large graphite flake](pictures/large_graphite.png)

## 5. Simulation of other molecules: 
Since some of the parameters for simulation engine reference to a specific group of molecules, to simulate different molecules, you need to make appropriate changes in mdp files.
In GromacsWrapper you can use it simple by importing command:

In [ ]:
from gromacs.cbook import edit_mdp

And then before using command gromacs.grompp() use edit_mdp command. For example, if you would like to simulate AL1 molecules, you can do it using one simple command:  

In [ ]:
edit_mdp('mdp/2_nvt.mdp', tc_grps=['GRAP', 'AL1', 'PHEO'],energygrps=['GRAP', 'AL1', 'PHEO'])

The coordination files for the different adsorbents are in "gro" subfolder.

References:
[1] Marrink, S. J., Risselada, H. J., Yefimov, S., Tieleman, D. P., and De Vries, A. H. (2007) The MARTINI force field: coarse grained model for biomolecular simulations. J. Phys. Chem. B 111, 7812–7824.
[2] Piskorz, T. K., Gobbo, C., Marrink, S.-J., Feyter, S. de, Vries, A. H. de,  Esch, J. H. van. Manuscript in preparation.
[3] Gobbo, C., Beurroies, I., De Ridder, D., Eelkema, R., Marrink, S. J., De Feyter, S.,  De Vries, A. H. (2013). MARTINI model for physisorption of organic molecules on graphite. Journal of Physical Chemistry C, 117(30), 15623-15631. 